In [6]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, Trainer
import pandas as pd
from multiprocessing import Pool
import re
import torch
from datasets import load_dataset
import numpy as np

2023-11-06 11:05:10.370299: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/donevas/opt/miniconda3/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [7]:
model_name = "dmis-lab/biobert-v1.1"
model_path = "./models/bert/trained/dmis-lab_biobert_v1.1/epochs_15_data_size_100_iter_2/"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_path)

In [9]:
model_name_short = "test"

In [16]:
def get_label_list(labels):
    print("*** returning unique labels ***")
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

def tokenize_and_align_labels(examples, tokenizer, text_column_name, label_column_name, label_to_id,
                              label_all_tokens=False, padding=True, max_length=512):
    print("*** Tokenize and Align Labels ***")
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=max_length,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    word_ids_list = []
    tokens_list = []
    for i, label in enumerate(examples[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        tokenized_tokens = tokenized_inputs.tokens(batch_index=i)
        word_ids_list.append(word_ids)
        previous_word_idx = None
        label_ids = []
        tokens_sub = [] # want to find out which of the tokens will be evaluated at the end, taking word-piece tokenization into account
        for i, word_idx in enumerate(word_ids):
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
                tokens_sub.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
                tokens_sub.append(tokenized_tokens[i])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label_to_id[label[word_idx]] if label_all_tokens else -100)
                tokens_sub.append(tokenized_tokens[i] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
        tokens_list.append(tokens_sub)
    #print(labels)
    #print(tokens_list)
    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = word_ids_list
    tokenized_inputs["tokenized_tokens"] = tokens_list
    return tokenized_inputs

In [20]:
def bert_predict_bio_format(model, tokenizer, ds_path_train, ds_path_test, text_column_name, label_column_name, label_all_tokens=False, padding=True, max_length=512):
    model = model
    data_files = {"train": ds_path_train,
                  "test": ds_path_test}
    raw_datasets = load_dataset("json", data_files=data_files)
    label_list = get_label_list(raw_datasets["train"][label_column_name])
    label_to_id = {l: i for i, l in enumerate(label_list)}
    id_to_label = {i: l for i, l in enumerate(label_list)}

    predict_dataset = raw_datasets["test"]
    predict_dataset = predict_dataset.map(
        tokenize_and_align_labels,
        batched=True,
        desc="Running tokenizer on prediction dataset",
        fn_kwargs={
            "tokenizer": tokenizer,
            "text_column_name": text_column_name,
            "label_column_name": label_column_name,
            "label_to_id": label_to_id,
            "label_all_tokens": label_all_tokens,
            "padding": padding,
            "max_length": max_length
        }
    )

    return predict_dataset_with_pred

In [23]:
ds_path_train = "./data/annotated_data/data_splits/ct_neuro_train_data_713.json"
ds_path_test = "./data/annotated_data/data_splits/ct_neuro_test_data_90.json"

ds_path_train = "./temp/corpus_files/ct_neuro_train_data_713.json"
ds_path_test = "./temp/corpus_files/ct_neuro_test_data_90.json"

text_column_name = "tokens"
label_column_name = "ner_tags"

In [24]:
bert_predict_bio_format(model, tokenizer, ds_path_train, ds_path_test, text_column_name, label_column_name)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

*** returning unique labels ***


Running tokenizer on prediction dataset:   0%|          | 0/90 [00:00<?, ? examples/s]

*** Tokenize and Align Labels ***


ArrowInvalid: Could not convert 'A' with type str: tried to convert to int64

In [25]:
data_files = {"train": ds_path_train,
              "test": ds_path_test}
raw_datasets = load_dataset("json", data_files=data_files)

In [26]:
predict_dataset = raw_datasets["test"]
train_dataset = raw_datasets["train"]

In [29]:
label_list = get_label_list(raw_datasets["train"][label_column_name])
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for i, l in enumerate(label_list)}

*** returning unique labels ***


In [31]:
text_column_name, label_column_name

('tokens', 'ner_tags')

In [39]:
def tokenize_and_align_labels(examples, tokenizer, text_column_name, label_column_name, label_to_id,
                              label_all_tokens=False, padding=True, max_length=512):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=max_length,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    word_ids_list = []
    for i, label in enumerate(examples[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        word_ids_list.append(word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label_to_id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = word_ids_list
    tokenized_inputs["tokenized_tokens"] = tokens_list
    return tokenized_inputs

In [43]:
def tokenize_and_align_labels(examples, tokenizer, text_column_name, label_column_name, label_to_id,
                              label_all_tokens=False, padding=True, max_length=512):
    print("*** Tokenize and Align Labels ***")
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=max_length,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    word_ids_list = []
    tokens_list = []
    for i, label in enumerate(examples[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        tokenized_tokens = tokenized_inputs.tokens(batch_index=i)
        word_ids_list.append(word_ids)
        previous_word_idx = None
        label_ids = []
        tokens_sub = [] # want to find out which of the tokens will be evaluated at the end, taking word-piece tokenization into account
        for i, word_idx in enumerate(word_ids):
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
                tokens_sub.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
                tokens_sub.append(tokenized_tokens[i])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label_to_id[label[word_idx]] if label_all_tokens else -100)
                tokens_sub.append(tokenized_tokens[i] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
        tokens_list.append(tokens_sub)

    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = word_ids_list
    #tokenized_inputs["tokenized_tokens"] = tokens_list
    return tokenized_inputs

In [44]:
train_dataset = train_dataset.map(
        tokenize_and_align_labels,
        batched=True,
        desc="Running tokenizer on train dataset",
        fn_kwargs={
            "tokenizer": tokenizer,
            "text_column_name": text_column_name,
            "label_column_name": label_column_name,
            "label_to_id": label_to_id,
            "label_all_tokens": False,
            "padding": True,
            "max_length": 512
        }
    )

Running tokenizer on train dataset:   0%|          | 0/713 [00:00<?, ? examples/s]

*** Tokenize and Align Labels ***
